# Referensi : https://www.youtube.com/watch?v=NBuED2PivbY


# Download chromeDriver first: https://chromedriver.chromium.org/home

In [ ]:
!pip install pillow
!pip install selenium
!pip install requests

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import io
from PIL import Image
import time
import os
import sys
from selenium.webdriver.common.keys import Keys

In [ ]:
def get_images_from_google(wd, googleimgsearch_url, delay, max_images):
    def scroll_down(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(delay)
        
    wd.get(googleimgsearch_url)

    image_urls = set()
    skips = 0
    
    reached_page_end = False
    last_height = wd.execute_script("return document.body.scrollHeight")
    
    while len(image_urls) + skips < max_images and not reached_page_end: 
        
        scroll_down(wd)
        new_height = wd.execute_script("return document.body.scrollHeight")
        if last_height == new_height:
            reached_page_end = True 
        else:
            last_height = new_height

        thumbnails = wd.find_elements(By.CLASS_NAME, "Q4LuWd")
        
        for img in thumbnails[len(image_urls) + skips:max_images]:
            try:
                img.click()
                time.sleep(delay)
            except:
                continue 

            images = wd.find_elements(By.CLASS_NAME, "iPVvYb")
            for image in images:
                if image.get_attribute('src') in image_urls:
                    max_images += 1
                    skips += 1
                    break

                if image.get_attribute('src') and 'http' in image.get_attribute('src'):
                    image_urls.add(image.get_attribute('src'))
                    sys.stdout.write("\rFound {0}".format(len(image_urls)))
                    sys.stdout.flush()

    return image_urls

In [ ]:
def get_images_from_tokopedia(wd, tokpedsearch_url, delay, max_images):
    def scroll_down(wd):
        element = wd.find_elements(By.CLASS_NAME, "css-llwpbs")[-1] # scroll down to this class element
        element.location_once_scrolled_into_view # alternative: wd.execute_script("arguments[0].scrollIntoView({ behavior: 'smooth', block: 'end', inline: 'end' });", element)
        time.sleep(delay)
    
    
    
    wd.get(tokpedsearch_url)
    
    last_height = wd.execute_script("return document.body.scrollHeight")
    
    image_urls = set()
    reached_page_end = False
    current_page = 1

    while len(image_urls) < max_images:
        
        # scrolling: find for thumbnails
        while not reached_page_end:
            scroll_down(wd)
            new_height = wd.execute_script("return document.body.scrollHeight")
            if last_height == new_height:
                reached_page_end = True 
            else:
                last_height = new_height
        
        # storing thumbnail links
        thumbnails = []
        thumbnails = wd.find_elements(By.CLASS_NAME, "css-llwpbs")
        thumbnails = set([thumb.find_element(By.XPATH,'.//a').get_attribute('href') for thumb in thumbnails])

        for thumb in thumbnails:
            
            wd.get(thumb); time.sleep(delay) # go to thumbnail link

            if len(wd.find_elements(By.CLASS_NAME, "active")) > 0: 
                wd.find_elements(By.CLASS_NAME, "active")[0].click(); # opening catalogues

                def find_image():
                        time.sleep(delay)
                        catalogues = wd.find_elements(By.CSS_SELECTOR, "img.css-46cmny.eit69wn2") # storing list of catalogue images
                        for cat in catalogues:
                            try:
                                cat.click()
                            except:
                                continue
                            image = wd.find_elements(By.CSS_SELECTOR, "img.css-udmgcf") # iterate through each catalogue image
                            image_urls.add(image[0].get_attribute('src')) # save each catalogue image URLs
                            sys.stdout.write("\rFound {0}".format(len(image_urls)))
                            sys.stdout.flush()

                find_image()

            wd.back() # back to previous page
 
        
        wd.find_element(By.TAG_NAME,'body').send_keys(Keys.CONTROL + Keys.HOME) # go to top of the page
        reached_page_end = False
        
        next_page_elem = wd.find_elements(By.CSS_SELECTOR, "button.css-16uzo3v-unf-pagination-item")
        current_scroll_position = 0
        scroll_speed = 8 # control scroll speed manually to slow it down
        
        while len(next_page_elem) <= 0 and not reached_page_end: # scrolling: find "next page" button
            wd.execute_script("window.scrollTo(0, {});".format(current_scroll_position))
            new_height = wd.execute_script("return document.body.scrollHeight")
            current_scroll_position += scroll_speed
            if last_height == new_height: 
                reached_page_end = True 
            else:
                last_height = new_height
        
        try:
            next_page_elem[1].click() # click next page button
            current_page += 1; print(f'Moving on to page {current_page}')
            reached_page_end = False
        except:
            print(f'\nError, can\'t move on to the next page.\nStopped at page {current_page}')
            break


    return image_urls    

In [ ]:
def download_image(url, download_path, file_name):
    try:
        image_content = requests.get(url).content
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file)
        file_path = download_path + file_name

        with open(file_path, "wb") as f:
            image.save(f, "JPEG")

    except Exception as e:
        print(f'\n{url} FAILED -', e) 

---
# Specify these variables

* "download_folder_name" : in which folder you want to save your downloaded images 
* "MAX_IMAGES" : set limit to maximum number of images that you want to download
* "googleimgsearch_url" : URL of google search image 
* "tokpedsearch_url" : URL of tokopedia search page

In [ ]:
MAX_IMAGES = 5

In [ ]:
#### >> FOR SCRAPING FROM GOOGLE
# download_folder_name = 'kalimantan_utara_google'
# googleimgsearch_url = "https://www.google.com/search?q=baju+adat+dayak+kalimantan+utara&tbm=isch&ved=2ahUKEwi86Zb24Yn_AhUhCLcAHZrgC4QQ2-cCegQIABAA&oq=baju+adat+dayak+kalimantan+utara&gs_lcp=CgNpbWcQAzIHCAAQExCABDoLCAAQgAQQsQMQgwE6BQgAEIAEOgQIABAeOgYIABAFEB46BggAEAgQHlD8CVjAKmCeLGgBcAB4AIABe4gBwAqSAQQyMy4xmAEAoAEBqgELZ3dzLXdpei1pbWfAAQE&sclient=img&ei=idBrZPyAHaGQ3LUPmsGvoAg&bih=601&biw=1280&rlz=1C1CHBD_idID1025ID1026"

In [ ]:
#### >> FOR SCRAPING FROM TOKOPEDIA
download_folder_name = 'kalimantan_utara_tokped'
tokpedsearch_url = "https://www.tokopedia.com/search?st=product&q=baju%20adat%20dayak%20kalimantan%20utara&srp_component_id=01.07.00.00&srp_page_id=&srp_page_title=&navsource="

In [ ]:
# this will set up directory path automatically

current_dir = os.getcwd()

CHROME_PATH = os.path.join(current_dir,'chromedriver.exe')
DOWNLOAD_PATH = os.path.join(current_dir, 
                       f'{download_folder_name}/')

if not os.path.exists(DOWNLOAD_PATH):
    os.makedirs(DOWNLOAD_PATH) 

---

In [ ]:
wd = webdriver.Chrome(CHROME_PATH)

#### >> SCRAPING FROM GOOGLE
# img_urls = get_images_from_google(wd, googleimgsearch_url, delay = 1, max_images=MAX_IMAGES) 

#### >> SCRAPING FROM TOKOPEDIA
img_urls = get_images_from_tokopedia(wd, tokpedsearch_url, delay = 3, max_images=MAX_IMAGES) 

wd.quit()

In [ ]:
# downloading images

for i, url in enumerate(img_urls):
    download_image(url, DOWNLOAD_PATH, str(i) + ".jpg")

In [ ]:
# save lists of image URLs
with open(f'{download_folder_name}_urls.txt','w') as f:
    f.write(str(img_urls))

# load ".txt" file containing lists of image URLs
# import ast
# with open(f'{download_folder_name}_urls.txt','r') as f:
#     old_img_urls = ast.literal_eval(f.read())